# 使い方

- 上から順にShift + Enterしていく。
- main部分のwordsに検索ワードのリストを入力してShift + Enter(入力するワード数に制限はない)


以上。

- このnotebookと同じディレクトリに"検索ワード1+検索ワード2+検索ワード3.csv"ができる。


## 注意


### chromedriver_binaryのバージョンについて
↓と思ったけど、以前会ったときできていたのでこれは多分大丈夫。

ターミナルから、
```
pip install chromedriver_binary
または
conda install chromedriver_binary
```
しただけだと動かなかった。
自分のchromeのバージョンが古かった。77。
よって自分の環境では
```
pip uninstall chromedriver_binary してから
pip install chromedriver_binary==77.0.3865.40

または

conda uninstall chromedriver_binary してから
conda install chromedriver_binary==77.0.3865.40
```
とする。
バージョンに指定する数値は

[スクレイピング時に「This version of ChromeDriver only supports Chrome version \.\.\.」のエラーが出た時 \- クラインの備忘壺](https://kleinblog.net/scraping-chrome-error/)

[Downloads \- ChromeDriver \- WebDriver for Chrome](http://chromedriver.chromium.org/downloads)

を参照。
 
 
 ### 取得数を増やしたいとき
 
 
 main部分の
 
 ```
 max_data_count = 2000
```

を増やす。



### 取得する属性を増やしたいとき

相談してください。

## 要件定義

欲しい情報

検索結果一覧のから各商品のページに移って(これを詳細ページと呼ぼう)、詳細ページの

右側にある

- 出品者の3段階評価マーク(顔のマーク)のそれぞれの個数
   ex.ニッコリマーク59個、真顔マーク1個、残念マーク0個
- 「商品の状態」欄のテキスト情報
   ex.新品、未使用
- 「配送料の負担」欄のテキスト情報
   ex.送料込み(出品者負担)
- 「配送の方法」欄のテキスト情報
   ex.らくらくメルカリ便
- 「配送元地域」欄のテキスト情報 
   ex.神奈川県

・値段
・販売中か売り切れたかのダミー
・左側にある掲載画像の枚数(写真自体ではなく枚数)
・値段の下にある商品説明のテキストデータ

これらのデータが欲しい。そして前回みたく検索ワードを変更出来る仕様、csvに落とし込んでくれる仕様


## 確認すべきこと

- 1件につき5秒、一覧一ページにつき5びょうかかるので1ページ100商品だと8分20秒かかる。 一覧ページは無尽蔵にあるようなので、100ページあるとすると13時間くらいかかる 。
- あまりに途方もなさすぎるので、取得数を限定する。


## その他開発にあたりメモ
    
    
テストすべきこと

-　成功 詳細一ページにつき各属性を要件通りに取得できるか→print
- 　成功　一覧一ページにつきdataframeをエラーなく作成できるか→isnull.max
- 成功　途中でブラウザが落ちないか
   

In [8]:
# ① ライブラリ読み込み部分
import requests
from bs4 import BeautifulSoup
import re
import os
import time
from time import sleep
import pandas as pd
from datetime import datetime
import traceback
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary

In [9]:
# soupから指定したセレクタを抜き出すが、存在しなかったらnullを返す関数

def select_from_soup(selector,soup):
    if soup.select(selector):
        target = soup.select(selector)[0].text
        return target
    else :
        return None
    

In [10]:
# ② 詳細ページから分析に必要な属性を抜き出して一覧ページに向けて返す関数

def get_detail_dictionary(link):
    driver.get(link)
    time.sleep(2) # ここをもっと少ない値にするともっと処理時間を短縮できるが、2秒は待ったほうが良い気がする。
    soup = BeautifulSoup(driver.page_source, "html.parser")        
    
    #ここから属性取得を開始する

    # 出品者の3段階評価マーク(顔のマーク)のそれぞれの個数   ex.ニッコリマーク59個、真顔マーク1個、残念マーク0個
#     good    = soup.select(".icon-good+span")[0].text 
    good = select_from_soup(".icon-good+span",soup)
    if good : 
        good    = int(good)                                                   #文字列型では分析できないので数値型に
#     normal =  soup.select(".icon-normal+span")[0].text
    normal = select_from_soup(".icon-normal+span",soup)
    if normal:
        normal = int(normal)
#     bad      = soup.select(".icon-bad+span")[0].text
    bad = select_from_soup(".icon-bad+span",soup)
    if bad:
        bad      = int(bad)

    # 「商品の状態」欄のテキスト情報   ex.新品、未使用
    # 4番目のtrに入っている
#     status  = soup.select(".item-detail-table tr:nth-child(4) td")[0].text
    status = select_from_soup(".item-detail-table tr:nth-child(4) td",soup)
    #    - 「配送料の負担」欄のテキスト情報   ex.送料込み(出品者負担)
    # 5番目のtrに入っている
#     charge  = soup.select(".item-detail-table tr:nth-child(5) td")[0].text
    charge = select_from_soup(".item-detail-table tr:nth-child(5) td",soup)

    #- 「配送の方法」欄のテキスト情報   ex.らくらくメルカリ便
    # ６番目
#     method = soup.select(".item-detail-table tr:nth-child(6) td")[0].text
    method = select_from_soup(".item-detail-table tr:nth-child(6) td",soup)

    # - 「配送元地域」欄のテキスト情報    ex.神奈川県
    # 7
#     region   = soup.select(".item-detail-table tr:nth-child(7) td")[0].text
    region = select_from_soup(".item-detail-table tr:nth-child(7) td",soup)

    # 値段
#     price = soup.select(".item-price.bold")[0].text.replace("¥","").replace(",","")
    price = select_from_soup(".item-price.bold",soup)
    if price: 
        price = price.replace("¥","").replace(",","")
        price = int(price)

    #販売中か売り切れたかのダミー
    #売り切れ商品のページには<div class="item-sold-out-badge"><div>SOLD</div></div>がある
    # 売り切れていたら1, 残っていたら0
    if soup.select(".item-sold-out-badge"):
        sold = 1
    else:
        sold = 0

    #   ・左側にある掲載画像の枚数(写真自体ではなく枚数)
    # 写真一枚は<div class="owl-dot-inner"> に入っている
    if soup.select(".owl-dot-inner"):
        image_count = len(soup.select(".owl-dot-inner"))
    else:
        image_count = None
    
    #・値段の下にある商品説明のテキストデータ
    # .item-description-inner　
#     description = soup.select(".item-description-inner")[0].text
    description = select_from_soup(".item-description-inner",soup)


    print(good,normal, bad, status, charge, method, region, price, sold, image_count, description[0:30])
#     print(normal)
#     print(bad)
#     print(status)
#     print(charge)
#     print(method)
#     print(region)
#     print(price)
#     print(sold)
#     print(image_count)
#     print(description[0:30])

    #データ名と値の辞書。詳細属性を追加したいときには上でスクレイピングしてここに追記すること。
    data_dictionary=  {"good": good,
           "normal":normal,
            "bad": bad,
            "status" : status,
            "charge": charge,
            "method": method, 
            "region": region,
            "price": price,
            "sold": sold, 
            "image_count": image_count,
           "description": description
#                        "timedelta": timedelta  　←追記の例
           }
    return data_dictionary


In [11]:
# ③検索ワードから一覧URLを生成する関数


def create_index_url(words):
    if (len(words) < 1):
        raise "検索ワードがありません。何か入力してください"
    words = list(map(lambda x : urllib.parse.quote(x), words))
    base_url = "https://www.mercari.com/jp/search/?keyword="
    return base_url + "%20".join(words)

# テスト
# 出力値がブラウザで直接検索したURLと完全に同一か： 同一なら　True
# 　成功
# create_index_url(["airpods","第二世代", "新品" ]) == "https://www.mercari.com/jp/search/?keyword=airpods+%E7%AC%AC%E4%BA%8C%E4%B8%96%E4%BB%A3+%E6%96%B0%E5%93%81"




In [13]:
# ●メイン部分




print("取得を開始します")
# url = "https://www.mercari.com/jp/search/?keyword=airpods+%E7%AC%AC2%E4%B8%96%E4%BB%A3+%E6%96%B0%E5%93%81"
options = Options()
driver = webdriver.Chrome(options=options)

# wordsに検索ワードを[リストで]入力
words =["airpods", "第二世代", "新品"]
url = create_index_url(words)
try:
    
     #各属性を1個ずつ、2000個収めるリストたち
    #前回も書いたが、2000回取得ののちリストを結合してDataFrameを作るのがもっとも高速。かつメモリの節約。

    good_list = []
    normal_list = []
    bad_list = []
    status_list = []
    charge_list = []
    method_list = []
    region_list = []
    price_list = []
    sold_list = []
    image_count_list = []
    description_list = []
        
        
    #２０００件集め終えたら一覧の取得ループも終わる
    has_next = True
    page_count = 1
    #一覧取得ループ : 
    # 次へが終わったらやめる
    while has_next :   
        print(f"一覧{page_count}ページ目取得開始...")
        url  = create_index_url(words)+ f"&page={page_count}"
#         url = create_index_url(["airpods", "第二世代","新品"])
        driver.get(url)
        time.sleep(5) #一覧は検索量が多く、5秒は待つのが吉。
        
        
#         スクロール処理(こうしないと要素が読み込まれない)
        height = driver.execute_script("return document.body.scrollHeight")
    #ループ処理で少しづつ移動
        print("ページスクロール中")
        for x in range(1,height,5):
            driver.execute_script("window.scrollTo(0, "+str(x)+");")
        time.sleep(4)
        print("ページスクロール終了")

        #一覧ページソース
        soup= BeautifulSoup(driver.page_source, "html.parser")
#         一覧ページに「該当する商品が見つかりません。検索条件を変えて、再度お試しください。」が見つかったら終わり
        if soup.select(".sc-cpmKsF"):
            print("一覧ページに「該当する商品が見つかりません」が表示されたので、取得を終了します。")
            break

        #一覧から商品詳細リンク95個取得し、(それぞれ<section class="items-box"> の下の aに入っている)
        # リンクタグのリストからhref属性、つまり商品詳細のurlのリストを取得
        href_list = []
#         for a in soup.select(".items-box>a"):
#         from IPython.core.debugger import Pdb; Pdb().set_trace()

        for a in soup.select(".sc-eMigcr a"):
            href_list.append(a.get("href"))
        print(f"一覧ページ{page_count}ページ目の{len(href_list)}個の詳細リンクをたどる")
        # 詳細取得ループ
        #取得したリンクたちに対してアクセスを繰り返す
        for  i, link in enumerate(href_list):
            print(f"一覧{page_count}ページ,詳細{i+1}商品目, 累計{len(good_list)+1}商品目を取得開始...")
            detail_dictionary = get_detail_dictionary(link) #上に関数定義。取得属性をいじりたいときは上のget_detail_dictionary関数に追記。
            
            # 取得した属性での
            good_list                .append(detail_dictionary["good"])
            normal_list             .append(detail_dictionary["normal"])
            bad_list                  .append(detail_dictionary["bad"])
            status_list             .append(detail_dictionary["status"])
            charge_list            .append(detail_dictionary["charge"])
            method_list          .append(detail_dictionary["method"])
            region_list             .append(detail_dictionary["region"])
            price_list               .append(detail_dictionary["price"])
            sold_list                .append(detail_dictionary["sold"])
            image_count_list.append(detail_dictionary["image_count"])
            description_list    .append(detail_dictionary["description"])
            print(f"一覧{page_count}ページ,詳細{i+1}商品目, 累計{len(good_list)}商品目を取得完了。 {link}")
            print("-"*100)
            
            #取得数が2000に達したら、
            #または「>」がなく次ページが取得できなかったら取得終わり
            max_data_count = 2000
            if len(good_list)>= max_data_count:
                print(f"詳細データ数が {max_data_count}に達したので取得を終了します。")
                has_next = False #こうすると一覧取得ループも終わる
                break
        page_count += 1


        # 詳細取得ループ終わり。
    #一覧取得ループ終わり。

    
     #dataframe作成
    df = pd.DataFrame()
    df["good"]             = good_list
    df["normal"]          = normal_list
    df["bad"]                = bad_list
    df["status"]           = status_list
    df["charge"]           = charge_list
    df["method"]         = method_list
    df["region"]            = region_list
    df["price_list"]       = price_list
    df["sold"]                = sold_list
    df["image_count"] = image_count_list
    df["description"]     = description_list
    
    df.to_csv("+".join(words)+".csv")
    print("dataframe書き出し完了。")
    
    print(df.head())
    print("全データ取得完了。お疲れ様でした。")
    
except:
    #何か失敗。エラー出力
    traceback.print_exc()
finally:
    # エラーが起きても起きなくてもブラウザを閉じる
    driver.quit()


取得を開始します
一覧1ページ目取得開始...
ページスクロール中
ページスクロール終了
一覧ページ1ページ目の98個の詳細リンクをたどる
一覧1ページ,詳細1商品目, 累計1商品目を取得開始...
14 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1480 0 8 大人気ダッフィーフレンズのステラルーの
Airpodsケース
一覧1ページ,詳細1商品目, 累計1商品目を取得完了。 https://item.mercari.com/jp/m68462128615/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細2商品目, 累計2商品目を取得開始...
192 6 1 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 山口県 19999 0 2 ★即日発送（早ければ数時間以内に即発送します！）
★匿名配送
一覧1ページ,詳細2商品目, 累計2商品目を取得完了。 https://item.mercari.com/jp/m21704805853/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細3商品目, 累計3商品目を取得開始...
15 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 奈良県 1600 1 4 新品未開封
2019年新型 Airpods第2世代   Ai
一覧1ページ,詳細3商品目, 累計3商品目を取得完了。 https://item.mercari.com/jp/m73185256180/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細4商品目, 累計4商品目を取得開始...
505 2 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 18999 1 

131 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 27699 1 6 アップル 純正品 エアーポッズ Apple AirPods 
一覧1ページ,詳細30商品目, 累計30商品目を取得完了。 https://item.mercari.com/jp/m71505335023/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細31商品目, 累計31商品目を取得開始...
63 1 1 新品、未使用 送料込み(出品者負担) 未定 山梨県 18000 1 2 ご覧頂きありがとうございます。
新品未開封  
Apple純
一覧1ページ,詳細31商品目, 累計31商品目を取得完了。 https://item.mercari.com/jp/m55637681331/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細32商品目, 累計32商品目を取得開始...
847 3 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 大分県 18600 1 3 新品未使用未開封

《純正品》MV7N2J/A 
AirPo
一覧1ページ,詳細32商品目, 累計32商品目を取得完了。 https://item.mercari.com/jp/m87347650697/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細33商品目, 累計33商品目を取得開始...
276 2 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 999 1 6 大人気モンスターズインクのAirpodsケースです★
サリー
一覧1ページ,詳細33商品目, 累計33商品目を取得完了。 https://item.mercari.com/jp/m67907

426 3 2 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 愛知県 1280 0 4 即購入♡歓迎です(ㆁᴗㆁ✿)

◯商品状態◯
自宅保管、新品
一覧1ページ,詳細59商品目, 累計59商品目を取得完了。 https://item.mercari.com/jp/m36718021806/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細60商品目, 累計60商品目を取得開始...
102 0 0 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 千葉県 1160 0 4 当製品は AirPods第二世代と第一世代両方にぴったり適用
一覧1ページ,詳細60商品目, 累計60商品目を取得完了。 https://item.mercari.com/jp/m75860673403/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細61商品目, 累計61商品目を取得開始...
1835 18 1 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 愛知県 18850 1 2 Apple アップル AirPods with Chargi
一覧1ページ,詳細61商品目, 累計61商品目を取得完了。 https://item.mercari.com/jp/m79698694285/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細62商品目, 累計62商品目を取得開始...
102 3 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 兵庫県 18500 1 4 カメラの調子が悪く若干曇っています。
新品未使用です。

2
一覧1ページ,詳細62商品目, 累計62商品目を取得完了。 https://item.mercari.com/jp

3 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 999 1 5 大人気モンスターズインクのAirpodsケースです‪(*ˊᵕ
一覧1ページ,詳細88商品目, 累計88商品目を取得完了。 https://item.mercari.com/jp/m46451075598/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細89商品目, 累計89商品目を取得開始...
3 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 999 1 4 大人気モンスターズインクのAirpodsケースです‪(*ˊᵕ
一覧1ページ,詳細89商品目, 累計89商品目を取得完了。 https://item.mercari.com/jp/m97630524206/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細90商品目, 累計90商品目を取得開始...
14 0 0 新品、未使用 送料込み(出品者負担) 未定 奈良県 680 1 7 商品説明


AirPods第二世代と第一世代両方にぴったり
一覧1ページ,詳細90商品目, 累計90商品目を取得完了。 https://item.mercari.com/jp/m93827541241/
----------------------------------------------------------------------------------------------------
一覧1ページ,詳細91商品目, 累計91商品目を取得開始...
174 5 3 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 大阪府 23980 1 2 【新品未開封】Air Pods 送料無料 エアーポッズ Ap
一覧1ページ,詳細91商品目, 累計91商品目を取得完了。 https://item.mercari.com/jp/m2254674345

52 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1480 0 6 ～～～～～～～～～～～～～～～～～～～～～

ステラルーAi
一覧2ページ,詳細19商品目, 累計117商品目を取得完了。 https://item.mercari.com/jp/m84925670514/
----------------------------------------------------------------------------------------------------
一覧2ページ,詳細20商品目, 累計118商品目を取得開始...
156 2 1 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 宮城県 750 1 6 airpods カバーairpodsケース airpods 
一覧2ページ,詳細20商品目, 累計118商品目を取得完了。 https://item.mercari.com/jp/m17002950232/
----------------------------------------------------------------------------------------------------
一覧2ページ,詳細21商品目, 累計119商品目を取得開始...
21 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1480 0 4 ダッフィーフレンズ：ステラルー
Airpods　ケース・カバ
一覧2ページ,詳細21商品目, 累計119商品目を取得完了。 https://item.mercari.com/jp/m20203531909/
----------------------------------------------------------------------------------------------------
一覧2ページ,詳細22商品目, 累計120商品目を取得開始...
123 1 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 19000 1 3 ソフトバンクのキャンペーンでいただいたものです、
新品未使用
一覧2ページ,詳細22商品目, 累計120商品目を取得完了。 https://item.mercari

...

85 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 愛知県 20799 1 5 Apple AirPods

ご覧いただきありがとうございま
一覧13ページ,詳細26商品目, 累計1043商品目を取得完了。 https://item.mercari.com/jp/m76729225453/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細27商品目, 累計1044商品目を取得開始...
698 6 1 新品、未使用 送料込み(出品者負担) ゆうゆうメルカリ便 愛知県 7700 1 None ※誠勝手ながらこちらのお品物はお値引きはできません

※20
一覧13ページ,詳細27商品目, 累計1044商品目を取得完了。 https://item.mercari.com/jp/m74290790856/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細28商品目, 累計1045商品目を取得開始...
44 1 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 大阪府 20350 1 2 Apple Airpods with charging ca
一覧13ページ,詳細28商品目, 累計1045商品目を取得完了。 https://item.mercari.com/jp/m57995683935/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細29商品目, 累計1046商品目を取得開始...
530 1 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 埼玉県 22100 1 3 新品・未使用品です。

アップル Apple
最新モデル　　
一覧13ページ,詳細29商品目, 累計1046商品目を取得完了。 https:

322 8 2 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 神奈川県 26000 1 3 Apple製
最高級品ワイヤレスイヤホン 最新型！！
Air
一覧13ページ,詳細54商品目, 累計1071商品目を取得完了。 https://item.mercari.com/jp/m59292146978/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細55商品目, 累計1072商品目を取得開始...
125 0 0 新品、未使用 送料込み(出品者負担) 普通郵便(定形、定形外) 福岡県 1280 1 8 対応機種: Apple AirPods 
（第二世代未検証 
一覧13ページ,詳細55商品目, 累計1072商品目を取得完了。 https://item.mercari.com/jp/m42481654141/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細56商品目, 累計1073商品目を取得開始...
43 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 24000 1 4 Apple アップル 純正 ワイヤレスイヤホン
AirPod
一覧13ページ,詳細56商品目, 累計1073商品目を取得完了。 https://item.mercari.com/jp/m36264677461/
----------------------------------------------------------------------------------------------------
一覧13ページ,詳細57商品目, 累計1074商品目を取得開始...
77 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 愛知県 25000 1 5 Apple製
最高級品ワイヤレスイヤホン 最新型
第2世代A
一覧13ページ,詳細57商品目, 累計1074商品目を取得完了。 https

54 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 21900 1 3 Apple AirPods 第2世代 MV7N2J/A 20
一覧14ページ,詳細3商品目, 累計1099商品目を取得完了。 https://item.mercari.com/jp/m18539667715/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細4商品目, 累計1100商品目を取得開始...
1127 6 2 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 長野県 65000 1 3 エアーポッズ 第2世代、ワイヤレス (ワイヤレス充電タイプで
一覧14ページ,詳細4商品目, 累計1100商品目を取得完了。 https://item.mercari.com/jp/m72235117901/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細5商品目, 累計1101商品目を取得開始...
54 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 22000 1 3 新品未開封
今月ヨドバシカメラにて購入した正規品です。
ワイ
一覧14ページ,詳細5商品目, 累計1101商品目を取得完了。 https://item.mercari.com/jp/m86819090814/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細6商品目, 累計1102商品目を取得開始...
161 9 1 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 23500 1 2 ◎ 正規品 Apple airpods 第二世代 MV7N2
一覧14ページ,詳細6商品目, 累計1102商品目を取得完了。 https://item.m

346 0 1 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 静岡県 27000 1 2 第二世代のairpods  ワイヤレス充電ケース付きです。

一覧14ページ,詳細31商品目, 累計1127商品目を取得完了。 https://item.mercari.com/jp/m81151407485/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細32商品目, 累計1128商品目を取得開始...
338 2 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 東京都 24999 1 2 アップルのAirPods 新型の正規品です。

発売したばか
一覧14ページ,詳細32商品目, 累計1128商品目を取得完了。 https://item.mercari.com/jp/m81191399998/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細33商品目, 累計1129商品目を取得開始...
161 2 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 大阪府 20777 1 4 4月26日まで！
売れなければ家人が使用致します！

★ 外
一覧14ページ,詳細33商品目, 累計1129商品目を取得完了。 https://item.mercari.com/jp/m42772624644/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細34商品目, 累計1130商品目を取得開始...
30 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 北海道 27399 1 2 Apple純正 AirPods 第2世代 with Wire
一覧14ページ,詳細34商品目, 累計1130商品目を取得完了。 https:/

47 1 1 新品、未使用 着払い(購入者負担) 未定 愛知県 23000 1 None ✴︎第2世代！新品！保証あり！✴︎アップル Airpods 
一覧14ページ,詳細59商品目, 累計1155商品目を取得完了。 https://item.mercari.com/jp/m56758617881/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細60商品目, 累計1156商品目を取得開始...
24 0 1 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 神奈川県 22222 1 4 (2019 新型) アップル  新型 Air pods 第2
一覧14ページ,詳細60商品目, 累計1156商品目を取得完了。 https://item.mercari.com/jp/m41391001723/
----------------------------------------------------------------------------------------------------
一覧14ページ,詳細61商品目, 累計1157商品目を取得開始...
97 0 0 新品、未使用 送料込み(出品者負担) らくらくメルカリ便 大阪府 17500 1 3 Apple  AirPodsです。
MMEF2J/A  
2
一覧14ページ,詳細61商品目, 累計1157商品目を取得完了。 https://item.mercari.com/jp/m78433095530/
----------------------------------------------------------------------------------------------------
一覧15ページ目取得開始...
ページスクロール中
ページスクロール終了
一覧ページに「該当する商品が見つかりません」が表示されたので、取得を終了します。
dataframe書き出し完了。
   good  normal  bad  status       charge        method region  pric

In [23]:
# # 商品名とページ番号nを与えると、商品一覧nページ目にある商品詳細情報を取得して一覧に返す
# # スクロールは手動で行うこととする


# # options = Options()
# # driver = webdriver.Chrome(options=options)

# words =["airpods", "第二世代", "新品"]
# page = 1
# url = create_index_url(words)
# try:
#     good_list = []
#     normal_list = []
#     bad_list = []
#     status_list = []
#     charge_list = []
#     method_list = []
#     region_list = []
#     price_list = []
#     sold_list = []
#     image_count_list = []
#     description_list = []
        
    
#     has_next = True
#     page_count = 1
#     #一覧取得ループ : 

#     print(f"一覧{page_count}ページ目取得開始...")
#     url = create_index_url(words)
#     driver.get(url)
# #     from IPython.core.debugger import Pdb; Pdb().set_trace()
#     #ここで手動スクロールする→いやだ。自動スクロールする
    
#     height = driver.execute_script("return document.body.scrollHeight")
#     #ループ処理で少しづつ移動
#     for x in range(1,height,5):
#         driver.execute_script("window.scrollTo(0, "+str(x)+");")
#     time.sleep(5) #一覧は検索量が多く、5秒は待つのが吉。
#     soup= BeautifulSoup(driver.page_source, "html.parser")

#     #一覧から商品詳細リンク95個取得し、(それぞれ<section class="items-box"> の下の aに入っている)
#     # リンクタグのリストからhref属性、つまり商品詳細のurlのリストを取得
#     href_list = []
# #         for a in soup.select(".items-box>a"):
    
#     for a in soup.select(".sc-eMigcr a"):
#         href_list.append(a.get("href"))
#     print(f"{len(href_list)}個のリンクをたどる")
#     for  i, link in enumerate(href_list):
#         print(f"一覧{page_count}ページ,詳細{i}商品目, 累計{len(good_list)+1}商品目を取得開始...")
#         detail_dictionary = get_detail_dictionary(link) #上に関数定義。取得属性をいじりたいときは上のget_detail_dictionary関数に追記。
            
#         # 取得した属性での
#         good_list                .append(detail_dictionary["good"])
#         normal_list             .append(detail_dictionary["normal"])
#         bad_list                  .append(detail_dictionary["bad"])
#         status_list             .append(detail_dictionary["status"])
#         charge_list            .append(detail_dictionary["charge"])
#         method_list          .append(detail_dictionary["method"])
#         region_list             .append(detail_dictionary["region"])
#         price_list               .append(detail_dictionary["price"])
#         sold_list                .append(detail_dictionary["sold"])
#         image_count_list.append(detail_dictionary["image_count"])
#         description_list    .append(detail_dictionary["description"])
#         print(f"一覧{page_count}ページ,詳細{i}商品目, 累計{len(good_list)}商品目を取得完了。 {link}")
#         print("-"*100)
            

    
#      #dataframe作成
#     df = pd.DataFrame()
#     df["good"]             = good_list
#     df["normal"]          = normal_list
#     df["bad"]                = bad_list
#     df["status"]           = status_list
#     df["charge"]           = charge_list
#     df["method"]         = method_list
#     df["region"]            = region_list
#     df["price_list"]       = price_list
#     df["sold"]                = sold_list
#     df["image_count"] = image_count_list
#     df["description"]     = description_list
    
#     os.makedirs("+".join(words),exist_ok=True)
#     df.to_csv("+".join(words)+f"{page}ページ.csv")
#     print("dataframe書き出し完了。")
    
#     print(df.head())
#     print("全データ取得完了。お疲れ様でした。")
    
# except:
#     #何か失敗。エラー出力
#     traceback.print_exc()
# finally:
#     # エラーが起きても起きなくてもブラウザを閉じる
#     driver.quit()


In [16]:
#　以下はテストなので実行する必要なし。

#　成功: テスト: Google検索ぺーじのaタグ取得()
# [Python \+ Selenium \+ ChromeでWebスクレイピングをやってみる \- Qiita](https://qiita.com/cl4nny_k/items/096ddae866a4886710e9)

# 成功: テスト: driver表示ページからSoupオブジェクトを取得
# driver.page_sourceに入っている
# options = Options()
# ヘッドレスモードで実行する場合
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

# try:
#     driver.get("https://www.google.co.jp/")
#     # 簡易的にJSが評価されるまで秒数で待つ
#     time.sleep(5)

#     #検索結果のページのHTMLをBeautifulSoupに流し込む
#     soup = BeautifulSoup(driver.page_source, "html.parser")
#     print(soup.select("a")[0])
# #     # aタグを抽出
# #     elem_list = driver.find_elements_by_tag_name("a")
# #     for elem in elem_list:
# #         # attributeの中からhrefを抽出して出力
# #         url = elem.get_attribute("href")
# #         print(url)

# except:
#     traceback.print_exc()
# finally:
#     # エラーが起きても起きなくてもブラウザを閉じる
#     driver.quit()

<a class="gb_e" data-pid="23" href="https://mail.google.com/mail/?tab=wm&amp;ogbl">Gmail</a>


In [37]:
#　レシーバって間を空けていいの？
# →　良い。
w = []
w   .   append("hello")
w


['hello']

In [16]:
d = pd.read_csv("airpods+第二世代+新品.csv",index_col=0)

#テスト：NaNがないか
# 成功：ない
# d.isnull().max()
d

,good,normal,bad,status,charge,method,region,price_list,sold,image_count,description
0,14,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,1480,0,8.0,大人気ダッフィーフレンズのステラルーの\nAirpodsケース\nケースを付けたまま充電もで...
1,192,6,1,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,山口県,19999,0,2.0,★即日発送（早ければ数時間以内に即発送します！）\n★匿名配送\n\n\n安心の配送事故補償...
2,15,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),奈良県,1600,1,4.0,新品未開封\n2019年新型 Airpods第2世代 Airpods2 新型Airpo...
3,505,2,0,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,東京都,18999,1,3.0,Apple MV7N2J/A Airpods with Charging Case 第2世代...
4,3,0,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,1260,0,6.0,大人気トイストーリーのリトルグリーンメン♪\nこちらはAirpodsケースになります(*ˊᵕ...
5,1239,2,0,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,大阪府,18500,1,3.0,AirPods with Charging Case 第二世代\n製品番号：MV7N2J/...
6,47,1,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),埼玉県,1000,1,NaN,AirPodsケース\n第一世代、第二世代に適用。\n新品未使用ですが箱などはありません。\...
7,276,2,0,新品、未使用,送料込み(出品者負担),普通郵便(定形、定形外),福岡県,999,0,7.0,※※こちらは訳ありB級品です※※\n\n\n ☆訳あり部分について☆\n\n詳細は...
8,674,10,2,新品、未使用,送料込み(出品者負担),未定,東京都,450,1,7.0,===================\n新品未使用\n全国送料無料 \nコ...
9,907,20,12,新品、未使用,送料込み(出品者負担),らくらくメルカリ便,北海道,19700,0,3.0,Apple MV7N2J/A Airpods with Charging Case \nAi...


In [ ]:
# テスト　breakの有効なスコープ

a = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"]]
abc = True
count = 0
while abc :
    arr = a[count]
    for i in arr:
        if i =="e":
            print("終了します。")
            break
        print(i)
    count += 1
    if count >=3:
        break
